In [1]:
from Algorithms.clarke_wright import ClarkeWright
from Algorithms.client import Client
import pandas as pd
from datetime import datetime, timedelta
import random
import names
import json
import numpy as np

# Generate historized client data

In [2]:

def generate_client_list(num_clients=50, start_date="2024-01-01", end_date="2024-12-31", 
                         min_slots=1, max_slots=3, locations=None):
    if locations is None:
        locations = ['Geldrop', 'Helmond', 'Someren', 'Deurne Vlierden', 'Mierlo']
    
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(start_date, date_format)
    end_date = datetime.strptime(end_date, date_format)
    date_range = (end_date - start_date).days

    timeslots = ['morning', 'evening']
    clients = []

    for _ in range(num_clients):
        name = names.get_full_name()  # Generating random full names
        location = random.choice(locations)
        
        # Generate appointment day
        appointment_day = start_date + timedelta(days=random.randint(0, date_range))
        appointment_hour = random.randint(0, 23)
        appointment_minute = random.randint(0, 59)
        appointment_time = datetime.combine(appointment_day, datetime.min.time()) + timedelta(hours=appointment_hour, minutes=appointment_minute)
        appointment_time_str = appointment_time.strftime(f"{date_format} %H:%M:%S")

        num_slots = random.randint(min_slots, max_slots)
        availability = []

        # Generate unique availability slots within 2 weeks after the appointment day or until end_date
        for _ in range(num_slots):
            while True:
                max_avail_day = min(appointment_day + timedelta(days=14), end_date)
                if max_avail_day <= appointment_day:
                    break
                day = appointment_day + timedelta(days=random.randint(1, (max_avail_day - appointment_day).days))
                timeslot = random.choice(timeslots)
                slot = f"{day.strftime(date_format)}_{timeslot}"

                # Ensure the new slot is unique
                if slot not in availability:
                    availability.append(slot)
                    break
        
        client = Client(name, location, availability, appointment_time_str)
        clients.append(client)

    return clients


config = {
    'num_clients': 50,
    'start_date': "2024-06-01",
    'end_date': "2024-06-07",
    'min_slots': 2,
    'max_slots': 2,
    'locations': ['Asten Heusden Ommel', 'Deurne Vlierden', 'Geldrop', 'Gemert Handel', 'Helmond',
                  'Helmond Brandevoort', 'Mierlo', 'Nuenen Gerwen Nederwetten', 'Someren'],
    'distance_matrix_path': "..//Data//distance_matrix.csv",
    'batch_size': 7  # For offline use case, processing batches of clients every 7 days
}

# random seed
random.seed(42)
 
# Example usage:
clients = generate_client_list(config['num_clients'], config['start_date'], config['end_date'], config['min_slots'],
                               config['max_slots'], config['locations'])
for i, client in enumerate(clients):
    print(f"Client {i + 1}: {client.name}, {client.location}, {client.availability}, {client.appointment_time}")

Client 1: James Lawrence, Gemert Handel, ['2024-06-07_evening', '2024-06-03_morning'], 2024-06-02 23:06:00
Client 2: Charles Whaley, Asten Heusden Ommel, ['2024-06-06_evening', '2024-06-07_morning'], 2024-06-05 06:45:00
Client 3: Travis Bowen, Geldrop, ['2024-06-06_morning', '2024-06-05_evening'], 2024-06-02 10:06:00
Client 4: Meghan Sartin, Someren, ['2024-06-07_evening', '2024-06-06_morning'], 2024-06-01 12:05:00
Client 5: John Murray, Helmond, ['2024-06-05_evening', '2024-06-03_evening'], 2024-06-01 07:55:00
Client 6: Kimberly Snyder, Deurne Vlierden, ['2024-06-06_evening', '2024-06-07_evening'], 2024-06-05 20:10:00
Client 7: Curtis Books, Asten Heusden Ommel, ['2024-06-06_evening', '2024-06-03_morning'], 2024-06-02 01:51:00
Client 8: Deborah Ponce, Nuenen Gerwen Nederwetten, ['2024-06-07_evening', '2024-06-05_morning'], 2024-06-02 08:08:00
Client 9: Carl Robinson, Asten Heusden Ommel, [], 2024-06-07 03:09:00
Client 10: Cathy Lamb, Nuenen Gerwen Nederwetten, ['2024-06-06_evening', '

# test online case

In [60]:
def get_definitive_timeslot_clarke(client, scheduled_definitive_appointments, distance_matrix_path):
    # print(f"1 Client {client.name} {client.location} {client.availability} {client.appointment_time}")
    # print(f"2 scheduled_definitive_appointments: {scheduled_definitive_appointments}")
    
    old_routes = []
    new_routes = []
    slots = [slot for slot in client.availability if slot in scheduled_definitive_appointments]
    # If the there are no appointments scheduled yet in the availability slots just schedule the client
    if not slots:
        # Schedule the client in the first available slot
        slot = client.availability[0]
        clarkewright = ClarkeWright([client])
        clarkewright.solve(slot, distance_matrix_path)
        scheduled_definitive_appointments[slot] = [[client], clarkewright.get_solution()]
        print(f"10000 scheduled_definitive_appointments: {scheduled_definitive_appointments}")
        return f"4 Client {client.name} has been scheduled for {slot}"
    
    # If there are appointments scheduled in the availability slots, try to add the client to the existing route with the minimal cost
    for slot in slots:
        # print(f"3 slot: {slot}")
        old_routes.append([scheduled_definitive_appointments[slot], slot])

        clients_new = scheduled_definitive_appointments[slot][0] + [client]
        clarkewright_new = ClarkeWright(clients_new)
        clarkewright_new.solve(slot, distance_matrix_path)
        new_routes.append([[clients_new, clarkewright_new.get_solution()], slot])

        
    # Compare the costs of all routes and pick the route with the smallest delta
    min_delta = float('inf')
    best_route = None
    for old_route, new_route in zip(old_routes, new_routes):
        old_cost = old_route[0][1][1]  # Extracting the cost from the old route
        new_cost = new_route[0][1][1]  # Extracting the cost from the new route
        delta = new_cost - old_cost
        if delta < min_delta:
            min_delta = delta
            best_route = new_route
    
    # Update the scheduled_definitive_appointments with the best route
    if best_route:
        slot = best_route[1]
    
        scheduled_definitive_appointments[slot] = [best_route[0][0], best_route[0][1]]
        print(f"200000 scheduled_definitive_appointments: {scheduled_definitive_appointments}")
        return f"Client {client.name} has been scheduled for {slot} with the minimal delta cost."


def online_use_case(clients, distance_matrix_path):
    scheduled_definitive_appointments = {}
    for client in sorted(clients, key=lambda x: x.appointment_time):
        message = get_definitive_timeslot_clarke(client, scheduled_definitive_appointments, distance_matrix_path)
        # print(message)
        

In [61]:
online_use_case(clients, config['distance_matrix_path'])

10000 scheduled_definitive_appointments: {'2024-06-02_morning': [[<Algorithms.client.Client object at 0x000001B25A2937F0>], (['Mierlo', 'Someren', 'Mierlo'], 154.0)]}
200000 scheduled_definitive_appointments: {'2024-06-02_morning': [[<Algorithms.client.Client object at 0x000001B25A2937F0>, <Algorithms.client.Client object at 0x000001B25A291090>], (['Mierlo', 'Someren', 'Helmond Brandevoort', 'Mierlo'], 225.0)]}
10000 scheduled_definitive_appointments: {'2024-06-02_morning': [[<Algorithms.client.Client object at 0x000001B25A2937F0>, <Algorithms.client.Client object at 0x000001B25A291090>], (['Mierlo', 'Someren', 'Helmond Brandevoort', 'Mierlo'], 225.0)], '2024-06-05_evening': [[<Algorithms.client.Client object at 0x000001B23A1C8100>], (['Mierlo', 'Helmond', 'Mierlo'], 148.0)]}
200000 scheduled_definitive_appointments: {'2024-06-02_morning': [[<Algorithms.client.Client object at 0x000001B25A2937F0>, <Algorithms.client.Client object at 0x000001B25A291090>, <Algorithms.client.Client object

IndexError: list index out of range

# test offline case

In [ ]:
def offline_use_case(clients, batch_size, distance_matrix_path):
    start_date = min(client.appointment_time for client in clients)
    end_date = max(client.appointment_time for client in clients)
    current_date = start_date

    appointments_data = []

    while current_date <= end_date:
        batch_clients = [client for client in clients if current_date <= client.appointment_time < current_date + timedelta(days=batch_size)]

        if batch_clients:
            for client in batch_clients:
                client.set_scheduled(current_date.strftime("%Y-%m-%d"))

            clarke = ClarkeWright(batch_clients)
            for client in batch_clients:
                selected_slots = client.availability
                user_data = {
                    "username": client.name,
                    "location": client.location
                }
                message, appointments_data = get_definitive_timeslot_clarke(selected_slots, user_data, appointments_data, distance_matrix_path)
                print(message)

        current_date += timedelta(days=batch_size)

In [ ]:
print("\nOffline Use Case Results:")
offline_use_case(clients, config['batch_size'], config['distance_matrix_path'])

In [ ]:
old_routes = 
[
    [
        [
            [<Algorithms.client.Client object at 0x000001A362F46EC0>, <Algorithms.client.Client object at 0x000001A303062980>], 
            [
                [<Algorithms.client.Client object at 0x000001A362F46EC0>, <Algorithms.client.Client object at 0x000001A303062980>], 
                (
                    ['Mierlo', 'Someren', 'Asten Heusden Ommel', 'Mierlo'], 225.0
                )
            ]
        ], '2024-06-07_evening'
    ]
]

new_routes = 
[
    [
        [
            [<Algorithms.client.Client object at 0x000001A362F46EC0>, <Algorithms.client.Client object at 0x000001A303062980>, <Algorithms.client.Client object at 0x000001A303036B00>], 
            (
                ['Mierlo', 'Someren', 'Asten Heusden Ommel', 'Nuenen Gerwen Nederwetten', 'Mierlo'], 310.0
            )
        ], 
        '2024-06-07_evening'
    ]
]